# Exploration du dataset NASA C-MAPSS

Ce notebook explore le dataset NASA C-MAPSS (Commercial Modular Aero-Propulsion System Simulation) utilisé pour l'entraînement des modèles de prédiction RUL.

## Dataset

- **Source**: NASA Prognostics Center of Excellence
- **Description**: Données de dégradation de moteurs turbofan
- **4 sous-datasets**: FD001, FD002, FD003, FD004
- **21 capteurs**: Températures, pressions, vitesses, débits
- **3 réglages opérationnels**

## Objectif

Prédire la Remaining Useful Life (RUL) en cycles avant défaillance.

In [ ]:
# Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

# Configuration plots
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
%matplotlib inline

# Largeur figures
plt.rcParams['figure.figsize'] = (14, 6)

## 1. Chargement des données

In [ ]:
# Chemins
DATA_DIR = Path('../data/raw/NASA_CMAPSS')

# Noms des colonnes
index_cols = ['unit_number', 'time_cycles']
setting_cols = ['setting_1', 'setting_2', 'setting_3']
sensor_cols = [f'sensor_{i}' for i in range(1, 22)]
col_names = index_cols + setting_cols + sensor_cols

# Charger FD001
train = pd.read_csv(
    DATA_DIR / 'train_FD001.txt',
    sep='\\s+',
    header=None,
    names=col_names
)

test = pd.read_csv(
    DATA_DIR / 'test_FD001.txt',
    sep='\\s+',
    header=None,
    names=col_names
)

rul_truth = pd.read_csv(
    DATA_DIR / 'RUL_FD001.txt',
    sep='\\s+',
    header=None,
    names=['RUL']
)

print(f"Train shape: {train.shape}")
print(f"Test shape: {test.shape}")
print(f"RUL truth shape: {rul_truth.shape}")

In [ ]:
# Aperçu des données
train.head()

In [ ]:
# Statistiques descriptives
train.describe()

## 2. Calcul de la RUL pour le training set

In [ ]:
# Pour chaque unit, la RUL = cycles max - cycle actuel
def add_rul(df):
    """Ajoute la colonne RUL au dataframe."""
    df = df.copy()
    
    # Calculer le cycle max pour chaque unit
    max_cycles = df.groupby('unit_number')['time_cycles'].max().reset_index()
    max_cycles.columns = ['unit_number', 'max_cycle']
    
    # Merge
    df = df.merge(max_cycles, on='unit_number', how='left')
    
    # Calculer RUL
    df['RUL'] = df['max_cycle'] - df['time_cycles']
    
    # Drop max_cycle
    df = df.drop('max_cycle', axis=1)
    
    return df

train = add_rul(train)

print(f"RUL range: {train['RUL'].min()} - {train['RUL'].max()}")
train[['unit_number', 'time_cycles', 'RUL']].head(10)

## 3. Analyse exploratoire

In [ ]:
# Distribution de la durée de vie des units
unit_lifetimes = train.groupby('unit_number')['time_cycles'].max()

plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.hist(unit_lifetimes, bins=30, edgecolor='black')
plt.xlabel('Cycles de vie')
plt.ylabel('Nombre d\'units')
plt.title('Distribution de la durée de vie des moteurs')
plt.axvline(unit_lifetimes.mean(), color='red', linestyle='--', label=f'Moyenne: {unit_lifetimes.mean():.0f}')
plt.legend()

plt.subplot(1, 2, 2)
plt.boxplot(unit_lifetimes)
plt.ylabel('Cycles de vie')
plt.title('Boxplot durée de vie')

plt.tight_layout()
plt.show()

print(f"Moyenne: {unit_lifetimes.mean():.2f} cycles")
print(f"Médiane: {unit_lifetimes.median():.2f} cycles")
print(f"Min: {unit_lifetimes.min()} cycles")
print(f"Max: {unit_lifetimes.max()} cycles")

In [ ]:
# Évolution des capteurs pour un moteur exemple
unit_example = 1
unit_data = train[train['unit_number'] == unit_example]

fig, axes = plt.subplots(4, 3, figsize=(16, 12))
axes = axes.flatten()

for i, sensor in enumerate(sensor_cols[:12]):
    axes[i].plot(unit_data['time_cycles'], unit_data[sensor])
    axes[i].set_xlabel('Cycles')
    axes[i].set_ylabel('Valeur')
    axes[i].set_title(f'{sensor}')
    axes[i].grid(True)

plt.suptitle(f'Évolution des capteurs - Unit {unit_example}', fontsize=14)
plt.tight_layout()
plt.show()

In [ ]:
# Corrélation entre capteurs
plt.figure(figsize=(14, 12))
correlation = train[sensor_cols].corr()
sns.heatmap(correlation, annot=False, cmap='coolwarm', center=0, vmin=-1, vmax=1)
plt.title('Matrice de corrélation des capteurs')
plt.tight_layout()
plt.show()

In [ ]:
# Variance des capteurs (identifier capteurs constants)
sensor_variance = train[sensor_cols].var().sort_values()

plt.figure(figsize=(12, 6))
sensor_variance.plot(kind='bar')
plt.ylabel('Variance')
plt.title('Variance par capteur')
plt.xticks(rotation=45)
plt.grid(axis='y')
plt.tight_layout()
plt.show()

print("\nCapteurs avec variance quasi-nulle (< 0.01):")
print(sensor_variance[sensor_variance < 0.01])

## 4. Évolution des capteurs en fonction de la RUL

In [ ]:
# Sélectionner des capteurs avec variance significative
significant_sensors = sensor_variance[sensor_variance > 0.01].index.tolist()[:6]

fig, axes = plt.subplots(2, 3, figsize=(16, 10))
axes = axes.flatten()

for i, sensor in enumerate(significant_sensors):
    # Moyenner par RUL (pour lisibilité)
    rul_grouped = train.groupby('RUL')[sensor].mean()
    
    axes[i].plot(rul_grouped.index, rul_grouped.values)
    axes[i].set_xlabel('RUL (cycles restants)')
    axes[i].set_ylabel('Valeur moyenne')
    axes[i].set_title(f'{sensor} vs RUL')
    axes[i].invert_xaxis()  # RUL décroît
    axes[i].grid(True)

plt.suptitle('Évolution des capteurs en fonction de la RUL', fontsize=14)
plt.tight_layout()
plt.show()

## 5. Feature engineering basique

In [ ]:
# Calculer rolling mean et rolling std sur fenêtre de 5 cycles
window = 5

def add_rolling_features(df, sensors, window=5):
    """Ajoute rolling mean et std pour les capteurs."""
    df = df.copy()
    
    for sensor in sensors:
        df[f'{sensor}_rolling_mean'] = df.groupby('unit_number')[sensor].transform(
            lambda x: x.rolling(window=window, min_periods=1).mean()
        )
        df[f'{sensor}_rolling_std'] = df.groupby('unit_number')[sensor].transform(
            lambda x: x.rolling(window=window, min_periods=1).std()
        )
    
    return df

# Appliquer sur quelques capteurs
train_fe = add_rolling_features(train, significant_sensors[:3], window=5)

print(f"Nouvelles features ajoutées:")
print([col for col in train_fe.columns if 'rolling' in col])

In [ ]:
# Visualiser l'effet du rolling mean
unit_example = 1
sensor_example = significant_sensors[0]
unit_data = train_fe[train_fe['unit_number'] == unit_example]

plt.figure(figsize=(14, 6))
plt.plot(unit_data['time_cycles'], unit_data[sensor_example], label='Original', alpha=0.6)
plt.plot(unit_data['time_cycles'], unit_data[f'{sensor_example}_rolling_mean'], 
         label=f'Rolling Mean (window={window})', linewidth=2)
plt.xlabel('Cycles')
plt.ylabel('Valeur')
plt.title(f'{sensor_example} - Original vs Rolling Mean (Unit {unit_example})')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

## 6. Préparation pour modélisation

### Clip RUL pour éviter l'instabilité

Souvent, on limite la RUL max à un seuil (ex: 125 cycles) car au-delà, l'équipement est considéré "sain" et la prédiction exacte est moins importante.

In [ ]:
# Clip RUL à 125
max_rul = 125
train_fe['RUL_clipped'] = train_fe['RUL'].clip(upper=max_rul)

plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.hist(train_fe['RUL'], bins=50, alpha=0.7, label='RUL originale')
plt.xlabel('RUL')
plt.ylabel('Fréquence')
plt.title('Distribution RUL originale')
plt.legend()

plt.subplot(1, 2, 2)
plt.hist(train_fe['RUL_clipped'], bins=50, alpha=0.7, label='RUL clippée', color='orange')
plt.xlabel('RUL')
plt.ylabel('Fréquence')
plt.title(f'Distribution RUL clippée (max={max_rul})')
plt.legend()

plt.tight_layout()
plt.show()

## 7. Sauvegarde des données préparées

In [ ]:
# Sauvegarder pour usage futur
output_dir = Path('../data/processed')
output_dir.mkdir(exist_ok=True)

train_fe.to_parquet(output_dir / 'train_FD001_features.parquet', index=False)
print(f"✓ Sauvegardé: {output_dir / 'train_FD001_features.parquet'}")
print(f"Shape: {train_fe.shape}")
print(f"Colonnes: {len(train_fe.columns)}")

## Conclusion

Ce notebook a permis de:
- ✅ Charger et explorer le dataset NASA C-MAPSS FD001
- ✅ Calculer la RUL pour le training set
- ✅ Analyser la distribution des durées de vie
- ✅ Identifier les capteurs informatifs
- ✅ Créer des features de base (rolling mean/std)
- ✅ Préparer les données pour modélisation

**Prochaines étapes**:
1. Feature extraction avancée (FFT, wavelets, kurtosis, etc.)
2. Entraînement modèle LSTM/GRU
3. Évaluation et tuning
4. Transfer learning vers données usine

Voir `02-rul-model-training.ipynb` pour la suite.